In [41]:
import docx
import csv
import pandas as pd

from simplify_docx import simplify

# read in a document 
my_doc = docx.Document("C:/Kuliah/Semester 8/SPF_IO/bahan_1_clear.docx")

# coerce to JSON using the standard options
my_doc_as_json = simplify(my_doc)

# or with non-standard options
my_doc_as_json_raw = simplify(my_doc,{"remove-leading-white-space":False})

In [42]:
material_book_1 = my_doc_as_json['VALUE'][0]['VALUE'].copy()

# List "paragraph_data" and "table" will contain dictonary that has structure like this
"""
    [
      {"TYPE":"paragraph","VALUE":
        {"TYPE":"paragraph","VALUE":"Kata Pengantar"}
      },
      {"TYPE":"paragraph","VALUE":
        {"TYPE":"paragraph","VALUE":"Syalom sejahtera bagi kita semua"}
      }
    ]
"""
paragraph_data = []
other_data=[]

# Open data dictionary of material_book_2 
for data_material in material_book_1 :
    if data_material['TYPE']=="paragraph":
        paragraph_data.append(data_material)
    else:
        other_data.append(data_material)

In [43]:
# Because the "paragraph_data" and "other_data" still contain dictionary inside the value.
# We need to get it and store it

# List "value_paragraph_data" will contain data like this:
"""
    [
      {"TYPE":"paragraph", 
       "VALUE":"Kata Pengantar",
       "style_type":"CT_Ind", 
       "style_left":"426", 
       "style_hanging":"426"},
       
      {"TYPE":"paragraph",
       "VALUE":"Syalom sejahtera bagi kita semua",
       "style_type":"CT_Ind",
       "style_left":"426",
       "style_hanging":"426"}
    ]
"""
value_paragraph_data = []
for data_paragraph in paragraph_data:
    dict_structure = data_paragraph['VALUE'][0]
    if "style" in data_paragraph.keys():
        if "indent" in data_paragraph['style'].keys():
            if "TYPE" in data_paragraph['style']['indent'].keys():
                dict_structure['style_type'] = data_paragraph['style']['indent']['TYPE']
            else:
                dict_structure['style_type'] = []
            if "left" in data_paragraph['style']['indent'].keys():
                dict_structure['style_left'] = data_paragraph['style']['indent']['left']  
            else:
                dict_structure['style_left'] = []
            if "hanging" in data_paragraph['style']['indent'].keys():
                dict_structure['style_hanging'] = data_paragraph['style']['indent']['hanging']        
            else:
                dict_structure['style_hanging'] = []
        else:
            dict_structure['style_type'] = []
            dict_structure['style_left'] = []
            dict_structure['style_hanging'] = []
    else:
        dict_structure['style_type'] = []
        dict_structure['style_left'] = []
        dict_structure['style_hanging'] = []
    
    if "style" in data_paragraph.keys():
        if "numPr" in data_paragraph['style'].keys():
            if "TYPE" in data_paragraph['style']['numPr'].keys():
                dict_structure['numPr_type'] = data_paragraph['style']['numPr']['TYPE']
            else:
                dict_structure['numPr_type'] = []
            if "ilvl" in data_paragraph['style']['numPr'].keys():
                dict_structure['numPr_ilvl'] = data_paragraph['style']['numPr']['ilvl'] 
            else:
                dict_structure['numPr_ilvl'] = []
            if "numId" in data_paragraph['style']['numPr'].keys():
                dict_structure['numPr_id'] = data_paragraph['style']['numPr']['numId']        
            else:
                dict_structure['numPr_id'] = []
        else:
            dict_structure['numPr_type'] = []
            dict_structure['numPr_ilvl'] = []
            dict_structure['numPr_id'] = [] 
    else:
        dict_structure['numPr_type'] = []
        dict_structure['numPr_ilvl'] = []
        dict_structure['numPr_id'] = [] 
        
    
        
    value_paragraph_data.append(dict_structure)



In [44]:
# Because the "paragraph_data" and "other_data" still contain dictionary inside the value.
# We need to get it and store it

# List "value_paragraph_data" will contain data like this:
"""
    [
      {"TYPE":"paragraph", 
       "VALUE":"Kata Pengantar"
      },
       
      {"TYPE":"paragraph",
       "VALUE":"Syalom sejahtera bagi kita semua"
      }
    ]
"""
value_other_data = []
for data_other in other_data:
    dict_structure_other = data_other['VALUE'][0]
    dict_structure_other_iter = data_other['VALUE']
    value_table_row=""
    for data_other_inside in dict_structure_other_iter:
        dict_structure_other_inside=data_other_inside["VALUE"]
        for data_other_inside_2 in dict_structure_other_inside:
            dict_structure_other_inside2 = data_other_inside_2['VALUE']
            value_table_col = ""
            try:
                for data_other_inside_3 in dict_structure_other_inside2:
                    data_structure_other_inside_3 = data_other_inside_3["VALUE"][0]['VALUE']
#                     print(data_structure_other_inside_3)
#                     print("=======================")
                    if data_structure_other_inside_3 not in value_table_row:
                        value_table_col+=data_structure_other_inside_3
                value_table_row+=value_table_col+" "
#                 print("======Row======")
#                 print(value_table_row)
            except:
#                 print("cell kosong")
                continue
    dict_structure_other['VALUE']=value_table_row
    value_other_data.append(dict_structure_other)


# Change directoary to CSV

### Data Table

In [46]:
# Chance Dictionary to CSV
col_name=["TYPE","VALUE"]
with open("material_book_1_other.csv", 'w', encoding="utf-8") as csvFileOther:
        wr = csv.DictWriter(csvFileOther, fieldnames=col_name)
        wr.writeheader()
        for ele in value_other_data:
            wr.writerow(ele)

### Data Paragraph

In [47]:
# Chance Dictionary to CSV
col_name=["TYPE","VALUE","style_type",'style_left','style_hanging','numPr_type','numPr_ilvl','numPr_id']
with open("material_book_1_paragraph.csv", 'w', encoding="utf-8") as csvFileOther:
        wr = csv.DictWriter(csvFileOther, fieldnames=col_name)
        wr.writeheader()
        for ele in value_paragraph_data:
            wr.writerow(ele)

In [48]:
# Load CSV paragraph
df_data_paragraph = pd.read_csv('material_book_2_paragraph.csv')

# Load CSV other
df_data_table = pd.read_csv('material_book_2_other.csv')

In [49]:
df_data_paragraph

,TYPE,VALUE,style_type,style_left,style_hanging,numPr_type,numPr_ilvl,numPr_id
0,text,Kata Pengantar,[],[],[],[],[],[]
1,text,"Tak terasa, kita sudah akan kembali menapaki M...",[],[],[],[],[],[]
2,text,Masa Adven-Natal 2020 ini sengaja mengambil te...,[],[],[],[],[],[]
3,text,Buku ini dapat terwujud sedemikian rupa karena...,[],[],[],[],[],[]
4,text,Pdt. Andreas Untung Wiyono (GKJ Tanjung Priok ...,CT_Ind,426,426,numPr,0,1
...,...,...,...,...,...,...,...,...
2620,text,Festival Digital Poster,CT_Ind,284,284,numPr,0,67
2621,text,Bagi anak-anak SMP diundang untuk mengikuti lo...,CT_Ind,284,[],[],[],[]
2622,text,Festival Lagu Natal Virtual,CT_Ind,284,284,numPr,0,67
2623,text,"Ditujukan bagi kaum muda-mudi. Kategori, penil...",CT_Ind,284,[],[],[],[]
